In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

In [ ]:
extracted = pd.read_excel(r'data/extracted_data.xlsx')
correction = pd.read_excel(r'data/agent_correction_requests.xlsx')

data = extracted.copy()
corrected = correction.copy()

In [203]:
class Data:
    def __init__(self, df1, df2):
        self.df1 = df1
        self.df2 = df2
    def clean_reference(self) -> pd.DataFrame:
        self.df1['Account Reference'] = self.df1['Account Reference'].str.replace('\'','')
        return self.df1

    def split_text(self) -> pd.DataFrame:
        note = self.df1.copy()
        notes = self.df1[self.df1['Notes'].str.len() > 100]
        notes.loc[:, 'Notes'] = notes.Notes.str[100:]
        note = pd.concat([note, notes])
        note.Notes = note.Notes.str[0:101]
        note = note.sort_index().reset_index(drop=True)
        return note

    def clean_data(self) -> pd.DataFrame:
        cleaned = self.df1.copy()
        corr = self.df2.copy().astype(str)
        corr['Date'] = corr['Date'] + " " + corr['Time']
        cleaned = cleaned[(cleaned['Date Gathered'].isin(corr['Date'])) & (cleaned['Account Reference'].isin(corr['Account number']))]

        for i in range(len(cleaned)):
            cleaned[corr.iloc[i,4]].iloc[i] = corr.iloc[i,5]
        self.df1.update(cleaned)
        return self.df1

    def correction_status(self) -> pd.DataFrame:
        cleaned = self.df1.copy()
        corr = self.df2.copy().astype(str)
        corr['Date'] = corr['Date'] + " " + corr['Time']
        cleaned = cleaned[(cleaned['Date Gathered'].isin(corr['Date'])) & (cleaned['Account Reference'].isin(corr['Account number']))]
        corr['Status'] = np.where(corr['Account number'].isin(cleaned['Account Reference']), "Corrected", "Not Found")
        return corr

    def save_data(self) -> pd.DataFrame:
        to_save = self.df1.copy()
        return to_save





In [202]:
mysample = Data(data, corrected)
mysample.clean_reference()
mysample.correction_status()


,Requester,Date,Time,Account number,Field to update,New value,Status
0,ANDREA,2024-09-12 09:44:36,09:44:36,OXXXXXXXPRTSTORX,Contact Source,OFFICE,Corrected
1,MARLON,2024-09-12 11:09:01,11:09:01,JYYYYYYYJFYMJMY6,Notes,09G6AA5RYB5 KOR 2X,Corrected
2,CHAZEL,2024-09-12 14:13:40,14:13:40,OXXXXXXXO9UPTT9P,Notes,09J9CT68QK4 DLR TT CH SD CLMPD 2K LST MNDY AND...,Corrected
3,ANDREA,2024-09-12 14:09:46,14:09:46,OXXXXXXXPXUOTSO8,Disposition Desc,FOLLOW UP,Corrected
4,RENZO,2024-09-12 15:36:40,15:36:40,OXXXXXXXPP8T9SRU,Notes,09F3AT453XX SKP KOR,Corrected
5,RENZO,2024-09-12 16:26:17,16:26:17,JYYYYYYYLYL76DMJ,Notes,09C5AR5892A SKP KOR,Corrected
6,CHAZEL,2024-09-12 16:43:08,16:43:08,10000000BFDFDIII,Notes,09ACVV6002F KOR,Not Found
